In [1]:
import pandas as pd

In [2]:
data = pd.read_csv(r"C:\Users\008846\Projects\Titanic Project\Titanic\train.csv")
test = pd.read_csv(r"C:\Users\008846\Projects\Titanic Project\Titanic\test.csv")
test_ids = test["PassengerId"]

def clean(data):
    data = data.copy()
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)
    
    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
         data[col] = data[col].fillna(data[col].median())
    
    data["Embarked"] = data["Embarked"].fillna("U")
    return data

data = clean(data)
test = clean(test)

In [3]:
data.head(5)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [4]:
from sklearn import preprocessing
la = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    la.fit(data[col])
    data[col] = la.fit_transform(data[col])
    test[col] = la.transform(test[col])
    print(la.classes_)

data.head(5)

['female' 'male']
['C' 'Q' 'S' 'U']


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [5]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split


y = data["Survived"]
x = data.drop("Survived", axis=1)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2, random_state=42)

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)  
x_val_scaled = scaler.transform(x_val)

In [7]:
clf = LogisticRegression(random_state=0, max_iter=1000)
clf.fit(x_train_scaled, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [8]:
from sklearn.metrics import accuracy_score
predictions = clf.predict(x_val_scaled)
accuracy = accuracy_score(y_val, predictions)
print(f"Model Accuracy: {accuracy:.2f}")

Model Accuracy: 0.81


In [9]:
test_scaled = scaler.transform(test)
submission_preds = clf.predict(test_scaled)

In [14]:
df = pd.DataFrame({"PassengerId":test_ids.values,
                   "Survived": submission_preds,
                  })
df.to_csv("Singhajitsubmission.csv", index=False)

In [13]:
import joblib
joblib.dump(clf, "titanic_survival_model.pkl")
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']